In [34]:
"""
import tensorflow as tf

# Construct a basic model.
root = tf.train.Checkpoint()
root.v1 = tf.Variable(3.)
root.v2 = tf.Variable(2.)
root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# Save the model.
export_dir = "/tmp/test_saved_model"
input_data = tf.constant(1., shape=[1, 1])
to_save = root.f.get_concrete_function(input_data)
tf.saved_model.save(root, export_dir, to_save)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()
"""

'\nimport tensorflow as tf\n\n# Construct a basic model.\nroot = tf.train.Checkpoint()\nroot.v1 = tf.Variable(3.)\nroot.v2 = tf.Variable(2.)\nroot.f = tf.function(lambda x: root.v1 * root.v2 * x)\n\n# Save the model.\nexport_dir = "/tmp/test_saved_model"\ninput_data = tf.constant(1., shape=[1, 1])\nto_save = root.f.get_concrete_function(input_data)\ntf.saved_model.save(root, export_dir, to_save)\n\n# Convert the model.\nconverter = tf.lite.TFLiteConverter.from_saved_model(export_dir)\ntflite_model = converter.convert()\n'

In [35]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
import os
from tensorflow.summary import FileWriter
import numpy as np
import glob
import scipy.io as spio

In [36]:
def output_tb:
    sess = tf.Session()
    tf.train.import_meta_graph("..\\ExpResults\\256x64\\model_512x32_lr_0_008_20200110.meta")
    FileWriter("__tb", sess.graph)

SyntaxError: invalid syntax (<ipython-input-36-7759025917dd>, line 1)

In [37]:
def convert2pb(meta_path,dir_name):
#     meta_path = 'model.ckpt-22480.meta' # Your .meta file
#     output_node_names = ['output:0']    # Output nodes
    
#     print(output_node_names)
    init=tf.global_variables_initializer()
    with tf.Session() as sess:
        # Restore the graph
        saver = tf.train.import_meta_graph(meta_path)

        # Load weights
        saver.restore(sess,tf.train.latest_checkpoint(dir_name))
        
        sess.run(init)
#         vars_train = tf.trainable_variables() 
#         output_node_names = [var.name.split(":")[0] for var in vars_train]
#         output_node_names = [n.name for n in tf.get_default_graph().as_graph_def().node]
#         print(output_node_names)
        graph_def = tf.get_default_graph().as_graph_def()
        
#         print(graph_def)
        # Freeze the graph
        frozen_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            graph_def,
            ['gradients_1/softmax_cross_entropy_with_logits_1_grad/LogSoftmax'])

        # Save the frozen graph
        with open('output_graph.pb', 'wb') as f:
          f.write(frozen_graph_def.SerializeToString())

In [38]:
def convert2tflite(saved_model_dir):
#     saved_model_dir = "../lite_models"
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
    tflite_model = converter.convert()
    open("512x32_0008.tflite", "wb").write(tflite_model)

In [39]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and return it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [40]:
if os.path.exists("../ExpResults/512x32_lr_0008/model_512x32_lr_0_008_20200110.meta"):
    print("file exist")
else:
    print("file does not exist")
# convert2pb("..\\ExpResults\\256x64\\model_512x32_lr_0_008_20200110.meta","..\\ExpResults\\256x64")
# tmp_graph = load_graph("..\\ExpResults\\256x64\\model_512x32_lr_0_008_20200110.meta")


file exist


In [41]:
def find_input_output_arrays(pb_file):
    gf = tf.GraphDef()   
    in_pb_file = open(pb_file,'rb')
    gf.ParseFromString(in_pb_file.read())

    with open('parameters.txt', 'a+') as the_file:
        for n in gf.node:
            the_file.write(n.name+'\n')

    file = open('parameters.txt','r')
    data = file.readlines()
    print("output name = ")
    print(data[len(data)-1])

    print("Input name = ")
    file.seek ( 0 )
    print(file.readline())

In [42]:
# find_input_output_arrays("../tensorflow_pb_files/512x32.pb")

In [43]:
def load_tflite(tflite_file=None):


    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path="../tensorflow_pb_files/512x32.tflite")
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    print(input_details)
    print(output_details)

#     # Test model on random input data.
#     input_shape = input_details[0]['shape']
#     input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
#     interpreter.set_tensor(input_details[0]['index'], input_data)

#     interpreter.invoke()

    # The function `get_tensor()` returns a copy of the tensor data.
    # Use `tensor()` in order to get a pointer to the tensor.
    input_data = interpreter.get_tensor(input_details[0]['index'])
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(input_data)
    print(output_data)

In [44]:
# load_tflite()

In [45]:
def tflite2CArray(pbfile):
    basic_model_size = os.path.getsize(pbfile)
    print("Basic model is %d bytes" % basic_model_size)
    quantized_model_size = os.path.getsize(pbfile)
    print("Quantized model is %d bytes" % quantized_model_size)
    difference = basic_model_size - quantized_model_size
    print("Difference is %d bytes" % difference)
    #****************************************************
    

In [46]:
tflite2CArray("../tensorflow_pb_files/512x32.tflite")

Basic model is 41112 bytes
Quantized model is 41112 bytes
Difference is 0 bytes


In [51]:
def getWeightFromTFLite(tflite_file):
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=tflite_file)
    interpreter.allocate_tensors()

    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()


    # get details for each layer
    all_layers_details = interpreter.get_tensor_details() 
    
#     for layer in all_layers_details:
#         print("layer name={}".format(layer['name']))
#         print("layer shape={}".format(layer['shape']))
#         data=interpreter.get_tensor(layer['index'])
#         print("data is: \n{}".format(data))
    
#     for layer in all_layers_details:
#         if layer['name'] == "Variable_6/transpose" \
#         or layer['name'] == "Variable_7/transpose" \
#         or layer['name']=="Variable_8/transpose" \
#         or layer['name']=="Relu_2" \
#         or layer['name']=="Relu_3" \
#         or layer['name']=="gradients_1/softmax_cross_entropy_with_logits_1_grad/LogSoftmax":
#             print("layer name={}".format(layer['name']))
#             print("layer shape={}".format(layer['shape']))
#             data=interpreter.get_tensor(layer['index'])
#             print("data is: \n{}".format(data))
     
    weight1 = []
    weight2 = []
    weight3 = []
    bias1 = []
    bias2= []
    bias3 = []
    for layer in all_layers_details:
        if layer['name']=="Relu_2":
            bias1 = interpreter.get_tensor(layer['index'])
        
        if layer['name']=="Relu_3":
            bias2 = interpreter.get_tensor(layer['index'])
            
        if layer['name']=="gradients_1/softmax_cross_entropy_with_logits_1_grad/LogSoftmax":
            bias3 = interpreter.get_tensor(layer['index'])
            
        if layer['name']=="Variable_6/transpose":
            w1_tranpose = np.array(interpreter.get_tensor(layer['index']))
            weight1 = w1_tranpose.T
            
        if layer['name']=="Variable_7/transpose":
            w2_tranpose = np.array(interpreter.get_tensor(layer['index']))
            weight2 = w2_tranpose.T
            
        if layer['name']=="Variable_8/transpose":
            w3_tranpose = np.array(interpreter.get_tensor(layer['index']))
            weight3 = w3_tranpose.T
            
    print("w1.shape = {},\n w2.shape = {},\n w3.shape = {}\n".format(weight1.shape, weight2.shape, weight3.shape))
    
    spio.savemat("testsavemat.mat",\
                 {'w1': weight1,'w2':weight2,'w3':weight3,\
                 'b1':bias1,'b2':bias2,'b3':bias3})
            
        
            
        


#     f = h5py.File("512x32_weights.hdf5", "w")   

#     for layer in all_layers_details:
#          # to create a group in an hdf5 file
#          grp = f.create_group(str(layer['index']))

#          # to store layer's metadata in group's metadata
#          grp.attrs["name"] = layer['name']
#          grp.attrs["shape"] = layer['shape']
#          # grp.attrs["dtype"] = all_layers_details[i]['dtype']
#          grp.attrs["quantization"] = layer['quantization']

#          # to store the weights in a dataset
#          grp.create_dataset("weights", data=interpreter.get_tensor(layer['index']))


#      f.close()

In [52]:
getWeightFromTFLite("../tensorflow_pb_files/512x32.tflite")

w1.shape = (40, 512),
 w2.shape = (512, 32),
 w3.shape = (32, 2)

